# 02 - Preprocessing YouTube Toxic Comments

En este notebook realizamos el **preprocesamiento** de los comentarios de YouTube
para dejar un CSV limpio y listo para el modelado posterior.

Objetivos:

- Cargar el dataset original usado en `01_EDA.ipynb`.
- Definir la columna de texto y las columnas de etiquetas.
- Crear una etiqueta binaria agregada (`IsAnyToxic`).
- Limpiar el texto con expresiones regulares.
- Tokenizar, eliminar *stopwords* y lematizar usando spaCy.
- Generar una columna de texto final `text_processed`.
- Guardar un CSV preprocesado para ser usado en `03_Modeling.ipynb`,
  donde se generarán los PKL (vectorizador, modelo, etc.).


## 1. Imports y configuración

Importamos las librerías necesarias:

- `pandas`, `numpy`: manejo de datos.
- `re`: expresiones regulares para limpieza de texto.
- `spacy`: tokenización y lematización.
- `pathlib`: manejo de rutas.

> **Nota:** Antes de ejecutar este notebook puede ser necesario instalar
> el modelo de spaCy con:  
> `python -m spacy download en_core_web_sm`


In [1]:
import warnings
warnings.filterwarnings("ignore")

import re
from pathlib import Path

import numpy as np
import pandas as pd

import spacy

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


## 2. Carga de datos

Cargamos el dataset original que ya hemos explorado en el notebook de EDA.

> Ajusta `DATA_PATH` a la ruta real donde tengas tu CSV.


In [5]:
# Ruta al CSV original (ajústala según tu estructura de proyecto)
DATA_PATH = Path("../../data/youtoxic_english_1000.csv")

df = pd.read_csv(DATA_PATH)

print("Tamaño del DataFrame (filas, columnas):", df.shape)
df.head()


Tamaño del DataFrame (filas, columnas): (1000, 15)


,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\r\nDont you reckon them 'black lives matter' ...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False,False,False,False


## 3. Definición de columnas de interés

Definimos:

- Columnas de ID (si existen) → `CommentId`, `VideoId` (ajusta si tu dataset difiere).
- Columna de texto → `Text`.
- Columnas de etiquetas → todas las columnas que empiecen por `"Is"`.
- Columna binaria agregada `IsAnyToxic` → 1 si alguna etiqueta de toxicidad es verdadera.

Esto nos permitirá:

- Trabajar en modo **binario** (tóxico vs no tóxico).
- O escalar a clasificación **multi-etiqueta** más adelante.


In [7]:
all_columns = df.columns.tolist()
print("Todas las columnas del dataset:")
print(all_columns)

# Ajusta estas columnas si en tu dataset tienen otros nombres
id_cols = [col for col in ["CommentId", "VideoId"] if col in df.columns]
text_col = "Text"

# Todas las columnas que empiezan por "Is" las consideramos etiquetas
label_cols = [c for c in df.columns if c.startswith("Is")]

print("\nColumnas de ID:", id_cols)
print("Columna de texto:", text_col)
print("Columnas de etiquetas:", label_cols)

# Columna binaria agregada
df["IsAnyToxic"] = df[label_cols].any(axis=1).astype(int)

df["IsAnyToxic"].value_counts(normalize=True).sort_index()


Todas las columnas del dataset:
['CommentId', 'VideoId', 'Text', 'IsToxic', 'IsAbusive', 'IsThreat', 'IsProvocative', 'IsObscene', 'IsHatespeech', 'IsRacist', 'IsNationalist', 'IsSexist', 'IsHomophobic', 'IsReligiousHate', 'IsRadicalism']

Columnas de ID: ['CommentId', 'VideoId']
Columna de texto: Text
Columnas de etiquetas: ['IsToxic', 'IsAbusive', 'IsThreat', 'IsProvocative', 'IsObscene', 'IsHatespeech', 'IsRacist', 'IsNationalist', 'IsSexist', 'IsHomophobic', 'IsReligiousHate', 'IsRadicalism']


IsAnyToxic
0    0.538
1    0.462
Name: proportion, dtype: float64

## 4. Manejo de valores nulos

Revisamos:

- Si hay textos nulos en `Text`.
- Si hay etiquetas nulas.

Acciones:

- Rellenar textos nulos con cadena vacía (`""`) para evitar errores de procesamiento.
- (Opcional) Eliminar filas completamente sin etiquetas, si existiesen.


In [8]:
print("Nulos en texto y etiquetas:\n")
print(df[[text_col] + label_cols + ["IsAnyToxic"]].isna().sum())

# Rellenar textos nulos con cadena vacía
df[text_col] = df[text_col].fillna("")

# (Opcional) Eliminar filas con todas las etiquetas NaN (no debería ser el caso)
# df = df.dropna(subset=label_cols, how="all").reset_index(drop=True)


Nulos en texto y etiquetas:

Text               0
IsToxic            0
IsAbusive          0
IsThreat           0
IsProvocative      0
IsObscene          0
IsHatespeech       0
IsRacist           0
IsNationalist      0
IsSexist           0
IsHomophobic       0
IsReligiousHate    0
IsRadicalism       0
IsAnyToxic         0
dtype: int64


## 5. Limpieza básica del texto (regex)

Creamos una función `clean_text` que:

- Convierte a minúsculas.
- Elimina URLs.
- Elimina menciones (`@user`).
- Sustituye hashtags (#hashtag) por la palabra sin `#`.
- Elimina tags HTML.
- Elimina caracteres no alfabéticos principales (dejamos letras y apóstrofes).
- Normaliza espacios múltiples.

Esta etapa reduce ruido antes de tokenizar y lematizar.


In [9]:
URL_PATTERN = re.compile(r"http\S+|www\.\S+")
MENTION_PATTERN = re.compile(r"@\w+")
HASHTAG_PATTERN = re.compile(r"#(\w+)")
HTML_TAG_PATTERN = re.compile(r"<.*?>")
NON_LETTER_PATTERN = re.compile(r"[^a-zA-Z' ]+")

def clean_text(text: str) -> str:
    """
    Limpia texto crudo aplicando varias transformaciones basadas en regex.
    """
    if not isinstance(text, str):
        text = str(text)

    # Minúsculas
    text = text.lower()

    # Quitar URLs
    text = URL_PATTERN.sub(" ", text)

    # Quitar menciones @user
    text = MENTION_PATTERN.sub(" ", text)

    # Sustituir hashtags por la palabra sin '#'
    text = HASHTAG_PATTERN.sub(r"\1", text)

    # Quitar tags HTML
    text = HTML_TAG_PATTERN.sub(" ", text)

    # Eliminar caracteres no alfabéticos principales (dejamos letras y apostrofes)
    text = NON_LETTER_PATTERN.sub(" ", text)

    # Normalizar espacios
    text = re.sub(r"\s+", " ", text).strip()

    return text


## 6. Aplicar la limpieza al texto

Aplicamos `clean_text` sobre la columna `Text` y creamos la columna `text_clean`.

Mostramos algunos ejemplos antes/después para verificar que la limpieza es razonable.


In [10]:
df["text_clean"] = df[text_col].apply(clean_text)

print("Ejemplos de limpieza de texto:")
df[[text_col, "text_clean"]].sample(5, random_state=RANDOM_STATE)


Ejemplos de limpieza de texto:


,Text,text_clean
521,You call yourself an anarchist but defend a co...,you call yourself an anarchist but defend a co...
737,My mother told me the same thing. God Bless t...,my mother told me the same thing god bless thi...
740,Love it I same the saem thing Go Peggy! #stup...,love it i same the saem thing go peggy stupid ...
660,"Next time they do that, line up some cars and ...",next time they do that line up some cars and s...
411,He was Robbing the Store and Being a Big Man ....,he was robbing the store and being a big man i...


## 7. Tokenización, stopwords y lematización con spaCy

En este paso:

- Cargamos el modelo de spaCy en inglés (`en_core_web_sm`).
- Definimos una función que:
  - Tokeniza.
  - Elimina stopwords, espacios y puntuación.
  - Lematiza cada token.
- Devolvemos un texto de salida con lemas concatenados, listo para
  ser vectorizado en el notebook de modelado.

> Si el modelo no está instalado, ejecuta en terminal:
> `python -m spacy download en_core_web_sm`


In [12]:
# Carga del modelo de spaCy
nlp = spacy.load("en_core_web_sm")

# Stopwords adicionales específicas del dominio (puedes ajustarlas tras el EDA)
custom_stopwords = {
    "video", "youtube", "watch", "channel",
    # añade aquí términos poco informativos que hayas visto en el EDA
}

def lemmatize_and_remove_stopwords(text: str) -> str:
    """
    Tokeniza, lematiza y elimina stopwords/puntuación usando spaCy.
    Devuelve un string con tokens lematizados, listo para vectorizar.
    """
    doc = nlp(text)

    tokens = []
    for token in doc:
        if token.is_space or token.is_punct:
            continue

        lemma = token.lemma_.lower().strip()

        # Stopwords de spaCy + stopwords personalizadas
        if lemma in nlp.Defaults.stop_words or lemma in custom_stopwords:
            continue

        if not lemma:
            continue

        tokens.append(lemma)

    return " ".join(tokens)


## 8. Generar la columna final `text_processed`

Aplicamos la función de lematización a `text_clean` para obtener `text_processed`.

Esta es la columna que usaremos en `03_Modeling.ipynb` para:

- Vectorizar (TF-IDF, embeddings, etc.).
- Entrenar los modelos de clasificación.


In [13]:
df["text_processed"] = df["text_clean"].apply(lemmatize_and_remove_stopwords)

print("Ejemplos de texto procesado:")
df[[text_col, "text_clean", "text_processed"]].sample(5, random_state=RANDOM_STATE)


Ejemplos de texto procesado:


,Text,text_clean,text_processed
521,You call yourself an anarchist but defend a co...,you call yourself an anarchist but defend a co...,anarchist defend cop shoot unarmed civilian hi...
737,My mother told me the same thing. God Bless t...,my mother told me the same thing god bless thi...,mother tell thing god bless woman
740,Love it I same the saem thing Go Peggy! #stup...,love it i same the saem thing go peggy stupid ...,love saem thing peggy stupid ya kill ya self q...
660,"Next time they do that, line up some cars and ...",next time they do that line up some cars and s...,time line car start burnout smoke riot gas non...
411,He was Robbing the Store and Being a Big Man ....,he was robbing the store and being a big man i...,rob store big man play fire burn police job


## 9. Comprobación rápida de etiquetas y texto procesado

Antes de guardar el CSV preprocesado, revisamos:

- Distribución de la etiqueta agregada `IsAnyToxic`.
- Ejemplos de texto procesado por clase (tóxico / no tóxico).

Esto sirve como sanity check del preprocesamiento.
